In [12]:
# 처음 한번만 받으면 됨
# !wget http://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth
!wget http://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r101_fpn_2x_coco/faster_rcnn_r101_fpn_2x_coco_bbox_mAP-0.398_20200504_210455-1d2dac9c.pth

--2021-05-14 03:08:49--  http://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r101_fpn_2x_coco/faster_rcnn_r101_fpn_2x_coco_bbox_mAP-0.398_20200504_210455-1d2dac9c.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 47.75.20.25
Connecting to download.openmmlab.com (download.openmmlab.com)|47.75.20.25|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 243518604 (232M) [application/octet-stream]
Saving to: ‘faster_rcnn_r101_fpn_2x_coco_bbox_mAP-0.398_20200504_210455-1d2dac9c.pth’

faster_rcnn_r101_fp 100%[===================>] 232.24M  9.33MB/s    in 30s     

2021-05-14 03:09:19 (7.82 MB/s) - ‘faster_rcnn_r101_fpn_2x_coco_bbox_mAP-0.398_20200504_210455-1d2dac9c.pth’ saved [243518604/243518604]



In [26]:
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)

In [27]:
# config file 들고오기
cfg = Config.fromfile('./configs/custom_augmentation/faster_rcnn_mosaic.py')
PREFIX = '../../input/data/'
# load
cfg.load_from = 'faster_rcnn_r50_fpn_2x_coco_bbox_mAP-0.384_20200504_210434-a5d8aa15.pth'

# dataset 바꾸기
cfg.data.train.ann_file = [PREFIX + 'train.json', PREFIX + 'pseudo_train.json']

cfg.work_dir = './work_dirs/faster_rcnn_2x_pseudo_train1'

print(f'Config:\n{cfg.pretty_text}')

Config:
model = dict(
    type='FasterRCNN',
    pretrained='torchvision://resnet50',
    backbone=dict(
        type='ResNet',
        depth=50,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=True,
        style='pytorch'),
    neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5),
    rpn_head=dict(
        type='RPNHead',
        in_channels=256,
        feat_channels=256,
        anchor_generator=dict(
            type='AnchorGenerator',
            scales=[8],
            ratios=[0.5, 1.0, 2.0],
            strides=[4, 8, 16, 32, 64]),
        bbox_coder=dict(
            type='DeltaXYWHBBoxCoder',
            target_means=[0.0, 0.0, 0.0, 0.0],
            target_stds=[1.0, 1.0, 1.0, 1.0]),
        loss_cls=dict(
            type='CrossEntropyLoss', use_sigmoid=True, loss_weight=1.0),
        loss_bbox=dict(typ

In [28]:
model = build_detector(cfg.model)


2021-05-17 06:33:14,733 - mmdet - INFO - load model from: torchvision://resnet50
2021-05-17 06:33:14,734 - mmdet - INFO - Use load_from_torchvision loader
2021-05-17 06:33:14,954 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias



In [30]:
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=0.92s)
creating index...
index created!


In [31]:
train_detector(model, datasets[0], cfg, distributed=False, validate=True)

loading annotations into memory...


2021-05-17 06:33:17,419 - mmdet - INFO - load checkpoint from ./work_dirs/faster_rcnn_batch8/epoch_12.pth
2021-05-17 06:33:17,419 - mmdet - INFO - Use load_from_local loader


Done (t=0.83s)
creating index...
index created!


2021-05-17 06:33:17,651 - mmdet - INFO - resumed epoch 12, iter 3936
2021-05-17 06:33:17,654 - mmdet - INFO - Start running, host: root@cca5f118b8f3, work_dir: /opt/ml/code/mmdetection_trash/work_dirs/faster_rcnn_1x_pseudo(vfnet)_resume_train
2021-05-17 06:33:17,654 - mmdet - INFO - workflow: [('train', 1)], max: 14 epochs
2021-05-17 06:33:49,965 - mmdet - INFO - Epoch [13][40/105]	lr: 2.000e-04, eta: -1 day, 23:26:17, time: 0.808, data_time: 0.074, memory: 8813, loss_rpn_cls: 0.4541, loss_rpn_bbox: 0.2442, loss_cls: 1.1215, acc: 75.1221, loss_bbox: 0.6348, loss: 2.4548, grad_norm: 7.4350
2021-05-17 06:34:19,971 - mmdet - INFO - Epoch [13][80/105]	lr: 2.000e-04, eta: -1 day, 23:26:57, time: 0.750, data_time: 0.014, memory: 8813, loss_rpn_cls: 0.1794, loss_rpn_bbox: 0.2384, loss_cls: 0.9565, acc: 75.3119, loss_bbox: 0.6212, loss: 1.9956, grad_norm: 3.2761
2021-05-17 06:34:38,883 - mmdet - INFO - Saving checkpoint at 13 epochs


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 655/655, 24.0 task/s, elapsed: 27s, ETA:     0s

...

Loading and preparing results...
DONE (t=0.91s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=7.51s).
Accumulating evaluation results...
DONE (t=1.70s).
Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.161
Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.289
Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.152
Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.054
Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.148
Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.288
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.359
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.359
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.359
Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ]

2021-05-17 06:35:51,520 - mmdet - INFO - Epoch [14][40/105]	lr: 2.000e-04, eta: -1 day, 23:31:33, time: 0.813, data_time: 0.073, memory: 8813, loss_rpn_cls: 0.1603, loss_rpn_bbox: 0.2344, loss_cls: 0.9016, acc: 76.1127, loss_bbox: 0.6124, loss: 1.9087, grad_norm: 2.6069
2021-05-17 06:36:21,820 - mmdet - INFO - Epoch [14][80/105]	lr: 2.000e-04, eta: -1 day, 23:30:08, time: 0.758, data_time: 0.014, memory: 8813, loss_rpn_cls: 0.1476, loss_rpn_bbox: 0.2313, loss_cls: 0.8899, acc: 76.3208, loss_bbox: 0.6066, loss: 1.8753, grad_norm: 2.4697
2021-05-17 06:36:40,618 - mmdet - INFO - Saving checkpoint at 14 epochs


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 655/655, 23.8 task/s, elapsed: 28s, ETA:     0s

2021-05-17 06:37:10,079 - mmdet - INFO - Evaluating bbox...


Loading and preparing results...
DONE (t=0.93s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=7.75s).
Accumulating evaluation results...
DONE (t=1.73s).
Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.169
Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.293
Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.166
Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.056
Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.158
Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.292
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.364
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.364
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.364
Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ]